In [1]:
import pandas as pd
import duckdb

In [2]:
Product = pd.DataFrame((
  ('Gizmo', 19.99, 'Gadgets', 'GizmoWorks'),
  ('Powergizmo', 29.99, 'Gadgets', 'GizmoWorks'),
  ('SingleTouch', 149.99, 'Photography', 'Canon'),
  ('MultiTouch', 203.99, 'Household', 'Hitachi'),
  ('Spinner', 39.99, 'Toy', 'Huawei'),
  ('SuperPhone', 399.99, 'Electronics', 'Huawei'),
  ('WindUpCar', 203.99, 'Toy', 'Xiaomi'),
  ('BatteryPack', 499.99, 'Electronics', 'BYD'),
), columns=('PName', 'Price', 'Category', 'Manufacturer'))
Product

,PName,Price,Category,Manufacturer
0,Gizmo,19.99,Gadgets,GizmoWorks
1,Powergizmo,29.99,Gadgets,GizmoWorks
2,SingleTouch,149.99,Photography,Canon
3,MultiTouch,203.99,Household,Hitachi
4,Spinner,39.99,Toy,Huawei
5,SuperPhone,399.99,Electronics,Huawei
6,WindUpCar,203.99,Toy,Xiaomi
7,BatteryPack,499.99,Electronics,BYD


In [3]:
Company = pd.DataFrame((
  ('GizmoWorks', 25, 'USA', 'Seattle'),
  ('Canon', 65, 'Japan', 'Chiba'),
  ('Hitachi', 15, 'Japan', 'Kyoto'),
  ('Huawei', 30, 'China', 'Shenzhen'),
  ('Xiaomi', 22, 'China', 'Xi\'an'),
  ('BYD', 100, 'China', 'Shenzhen'),
), columns=('CName', 'StockPrice', 'Country', 'City'))
Company

,CName,StockPrice,Country,City
0,GizmoWorks,25,USA,Seattle
1,Canon,65,Japan,Chiba
2,Hitachi,15,Japan,Kyoto
3,Huawei,30,China,Shenzhen
4,Xiaomi,22,China,Xi'an
5,BYD,100,China,Shenzhen


In [4]:
Purchase = pd.DataFrame((
  (0, 'MultiTouch', 'Joe Blow'),
  (1, 'Spinner', 'Joe Blow'),
  (2, 'SuperPhone', 'Joe Blow'),
  (3, 'BatteryPack', 'Joe Blow'),
  (4, 'Gizmo', 'Don Juan'),
  (5, 'WindUpCar', 'Rick Astley'),
  (6, 'Spinner', 'Rick Astley'),
  (7, 'Powergizmo', 'Don Juan'),
), columns=('id', 'Product', 'Buyer'))
Purchase

,id,Product,Buyer
0,0,MultiTouch,Joe Blow
1,1,Spinner,Joe Blow
2,2,SuperPhone,Joe Blow
3,3,BatteryPack,Joe Blow
4,4,Gizmo,Don Juan
5,5,WindUpCar,Rick Astley
6,6,Spinner,Rick Astley
7,7,Powergizmo,Don Juan


In [5]:
duckdb.query('''
  SELECT * FROM Product WHERE Manufacturer <> 'Canon'
''')

┌─────────────┬────────┬─────────────┬──────────────┐
│    PName    │ Price  │  Category   │ Manufacturer │
│   varchar   │ double │   varchar   │   varchar    │
├─────────────┼────────┼─────────────┼──────────────┤
│ Gizmo       │  19.99 │ Gadgets     │ GizmoWorks   │
│ Powergizmo  │  29.99 │ Gadgets     │ GizmoWorks   │
│ MultiTouch  │ 203.99 │ Household   │ Hitachi      │
│ Spinner     │  39.99 │ Toy         │ Huawei       │
│ SuperPhone  │ 399.99 │ Electronics │ Huawei       │
│ WindUpCar   │ 203.99 │ Toy         │ Xiaomi       │
│ BatteryPack │ 499.99 │ Electronics │ BYD          │
└─────────────┴────────┴─────────────┴──────────────┘

Slide 19

In [6]:
duckdb.query('''
  SELECT *
  FROM Product, Company
  WHERE Manufacturer = CName
  -- AND Country = 'Japan'
  -- AND Price < 200
''')

┌─────────────┬────────┬─────────────┬──────────────┬────────────┬────────────┬─────────┬──────────┐
│    PName    │ Price  │  Category   │ Manufacturer │   CName    │ StockPrice │ Country │   City   │
│   varchar   │ double │   varchar   │   varchar    │  varchar   │   int64    │ varchar │ varchar  │
├─────────────┼────────┼─────────────┼──────────────┼────────────┼────────────┼─────────┼──────────┤
│ Gizmo       │  19.99 │ Gadgets     │ GizmoWorks   │ GizmoWorks │         25 │ USA     │ Seattle  │
│ Powergizmo  │  29.99 │ Gadgets     │ GizmoWorks   │ GizmoWorks │         25 │ USA     │ Seattle  │
│ SingleTouch │ 149.99 │ Photography │ Canon        │ Canon      │         65 │ Japan   │ Chiba    │
│ MultiTouch  │ 203.99 │ Household   │ Hitachi      │ Hitachi    │         15 │ Japan   │ Kyoto    │
│ Spinner     │  39.99 │ Toy         │ Huawei       │ Huawei     │         30 │ China   │ Shenzhen │
│ SuperPhone  │ 399.99 │ Electronics │ Huawei       │ Huawei     │         30 │ China   │ S

Slide 20

In [7]:
duckdb.query('''
  SELECT DISTINCT CName, p.*, p2.*
  FROM Product AS p, Product as p2, Company AS c
  WHERE c.Country = 'China'
  AND c.CName = p.Manufacturer AND p.Category = 'Electronics'
  AND c.CName = p2.Manufacturer AND p2.Category = 'Toy'
''')

┌─────────┬────────────┬────────┬─────────────┬──────────────┬─────────┬────────┬──────────┬──────────────┐
│  CName  │   PName    │ Price  │  Category   │ Manufacturer │  PName  │ Price  │ Category │ Manufacturer │
│ varchar │  varchar   │ double │   varchar   │   varchar    │ varchar │ double │ varchar  │   varchar    │
├─────────┼────────────┼────────┼─────────────┼──────────────┼─────────┼────────┼──────────┼──────────────┤
│ Huawei  │ SuperPhone │ 399.99 │ Electronics │ Huawei       │ Spinner │  39.99 │ Toy      │ Huawei       │
└─────────┴────────────┴────────┴─────────────┴──────────────┴─────────┴────────┴──────────┴──────────────┘

Slide 21

In [8]:
duckdb.query('''
             
  SELECT DISTINCT CName, p1.*, p2.*
             -- DISTINCT p1.Manufacturer
  FROM Product AS p1, Product AS p2, Company AS c
  WHERE p1.Manufacturer = c.CName AND p2.Manufacturer = c.CName
  AND Country = 'China'
  AND (p1.Category = 'Toy'
  AND p2.Category = 'Electronics')
  -- ลองทำ
''')

┌─────────┬─────────┬────────┬──────────┬──────────────┬────────────┬────────┬─────────────┬──────────────┐
│  CName  │  PName  │ Price  │ Category │ Manufacturer │   PName    │ Price  │  Category   │ Manufacturer │
│ varchar │ varchar │ double │ varchar  │   varchar    │  varchar   │ double │   varchar   │   varchar    │
├─────────┼─────────┼────────┼──────────┼──────────────┼────────────┼────────┼─────────────┼──────────────┤
│ Huawei  │ Spinner │  39.99 │ Toy      │ Huawei       │ SuperPhone │ 399.99 │ Electronics │ Huawei       │
└─────────┴─────────┴────────┴──────────┴──────────────┴────────────┴────────┴─────────────┴──────────────┘

Slide 26

In [9]:
duckdb.query('''
  SELECT c.City
  FROM Company AS c
  WHERE c.CName IN (
    SELECT p.Manufacturer --, *
    FROM Purchase, Product AS p
    WHERE p.PName = Purchase.Product
    AND Purchase.buyer = 'Joe Blow'
  )
''')

┌──────────┐
│   City   │
│ varchar  │
├──────────┤
│ Kyoto    │
│ Shenzhen │
│ Shenzhen │
└──────────┘

Slide 27

In [10]:
duckdb.query('''
  SELECT *--c.City
  FROM Company AS c, Product AS p, Purchase
  WHERE c.CName = p.Manufacturer
  AND p.PName = Purchase.Product
  AND Purchase.buyer = 'Joe Blow'
''')

┌─────────┬────────────┬─────────┬──────────┬───┬─────────────┬──────────────┬───────┬─────────────┬──────────┐
│  CName  │ StockPrice │ Country │   City   │ … │  Category   │ Manufacturer │  id   │   Product   │  Buyer   │
│ varchar │   int64    │ varchar │ varchar  │   │   varchar   │   varchar    │ int64 │   varchar   │ varchar  │
├─────────┼────────────┼─────────┼──────────┼───┼─────────────┼──────────────┼───────┼─────────────┼──────────┤
│ Hitachi │         15 │ Japan   │ Kyoto    │ … │ Household   │ Hitachi      │     0 │ MultiTouch  │ Joe Blow │
│ Huawei  │         30 │ China   │ Shenzhen │ … │ Electronics │ Huawei       │     2 │ SuperPhone  │ Joe Blow │
│ BYD     │        100 │ China   │ Shenzhen │ … │ Electronics │ BYD          │     3 │ BatteryPack │ Joe Blow │
│ Huawei  │         30 │ China   │ Shenzhen │ … │ Toy         │ Huawei       │     1 │ Spinner     │ Joe Blow │
├─────────┴────────────┴─────────┴──────────┴───┴─────────────┴──────────────┴───────┴─────────────┴────

California Housing Train

In [13]:
Housing = pd.read_csv('C:\\Users\\Asus\\OneDrive\\Desktop\\IcyAlmondE\\RAC\\16-25 Oct\\california_housing_train.csv')

In [14]:
Housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')

In [15]:
duckdb.query('''
  SELECT AVG(total_bedrooms) FROM Housing
''')

┌─────────────────────┐
│ avg(total_bedrooms) │
│       double        │
├─────────────────────┤
│   539.4108235294118 │
└─────────────────────┘

In [19]:
duckdb.query('''
    SELECT housing_median_age, median_income
    FROM Housing
    --GROUP BY housing_median_age
''')

┌────────────────────┬───────────────┐
│ housing_median_age │ median_income │
│       double       │    double     │
├────────────────────┼───────────────┤
│               15.0 │        1.4936 │
│               19.0 │          1.82 │
│               17.0 │        1.6509 │
│               14.0 │        3.1917 │
│               20.0 │         1.925 │
│               29.0 │        3.3438 │
│               25.0 │        2.6768 │
│               41.0 │        1.7083 │
│               34.0 │        2.1782 │
│               46.0 │        2.1908 │
│                 ·  │           ·   │
│                 ·  │           ·   │
│                 ·  │           ·   │
│               40.0 │        1.9375 │
│               36.0 │        2.5434 │
│               29.0 │        1.2176 │
│               43.0 │        1.5174 │
│               43.0 │         1.869 │
│               44.0 │        1.7083 │
│               39.0 │        1.5227 │
│               51.0 │        3.3125 │
│               50.0 │   

In [44]:
duckdb.query('''
    --SELECT income_group, COUNT(*)
    --FROM
    --(
        SELECT 
        CASE
            WHEN median_income < 5 THEN 'L'
            WHEN median_income < 10 THEN 'M'
            ELSE 'H'
            END AS income_group,
        MIN(housing_median_age), MAX(housing_median_age), AVG(total_bedrooms), SUM(population),
        FROM Housing
        
    --)
    GROUP BY income_group
    HAVING AVG(total_bedrooms)>500
''')

┌──────────────┬─────────────────────────┬─────────────────────────┬─────────────────────┬─────────────────┐
│ income_group │ min(housing_median_age) │ max(housing_median_age) │ avg(total_bedrooms) │ sum(population) │
│   varchar    │         double          │         double          │       double        │     double      │
├──────────────┼─────────────────────────┼─────────────────────────┼─────────────────────┼─────────────────┤
│ L            │                     1.0 │                    52.0 │   543.2989799773328 │      18870946.0 │
│ M            │                     1.0 │                    52.0 │   536.2413694721826 │       5158848.0 │
└──────────────┴─────────────────────────┴─────────────────────────┴─────────────────────┴─────────────────┘

In [66]:
duckdb.query('''
    SELECT  COUNT(*),
            --MIN(housing_median_age), 
            AVG(housing_median_age), 
            --MAX(housing_median_age),
            --MIN(total_rooms), 
            AVG(total_rooms), 
            --MAX(total_rooms),
            --MIN(total_bedrooms), 
            AVG(total_bedrooms), 
            --MAX(total_bedrooms),
            --MIN(population), 
            AVG(population), 
            --MAX(population),
            --MIN(households), 
            AVG(households), 
            --MAX(households),
            --MIN(median_income), 
            AVG(median_income), 
            --MAX(median_income),
            --MIN(median_house_value), 
            AVG(median_house_value), 
            --MAX(median_house_value)
    FROM Housing AS h
    WHERE (h.latitude-34.42895833453776)**2 + (h.longitude- -119.7077359512203)**2 <= 0.01**2
''').df()

,count_star(),avg(housing_median_age),avg(total_rooms),avg(total_bedrooms),avg(population),avg(households),avg(median_income),avg(median_house_value)
0,14,40.857143,1497.285714,435.857143,851.357143,404.071429,2.700771,296000.0
